# Motivation
论文原文：[《Attentional Factorization Machines: Learning the Weight of Feature Interactions via Attention Networks》](http://www.ijcai.org/proceedings/2017/435)

参考学习地址：[推荐系统系列（八）：AFM理论与实践](https://zhuanlan.zhihu.com/p/94009156)

AFM的研究人员中包含了NFM作者，所以AFM中能明显看到NFM的影子。AFM贡献在于将attention机制引入到特征交叉模块，不同于NFM在Bi-Interaction的处理，AFM在对二阶交叉特征进行sum pooling时，使用attention network计算得到的attention score对交叉特征进行加权求和

$$
\begin{align}
\hat{y}_{AFM}(X)=w_0+\Sigma_{i=1}^nw_ix_i+p^T\Sigma_{i=1}^n\Sigma_{j=i+1}^na_{i,j}(v_i\odot v_j)x_ix_j
\end{align}
$$

作为对比 $f_{BI}(V_x)=\Sigma_{i=1}^n\Sigma_{j=i+1}^nx_iv_i\odot x_jv_j$，除去$p^T$代表prediction layer的weight，核心差异就是$a_{i,j}$项，而这部分代表的就是PI相对BI（或者说AFM相对于NFM）的优化：<font color="red">**Attention-based Pooling**</font>。

 <img style="display: block; margin: 0 auto;" src="../../../assets/images/afm.png" width = "1000" height = "400" alt="AFM" align=center />

## Pair-wise Interaction Layer
这部分工作主要将特征进行二阶交叉，$n$个特征两两交叉后得到$\frac{n(n-1)}{2}$个二阶交叉特征向量，有

$$
\begin{align}
f_{PI}(\xi)=\{(v_i\odot v_j)x_ix_j\}_{(i,j)\in \R_x}
\end{align}
$$

其中，$\xi$表示Embedding Layer的输出，$\R_x$表示所有可能的二阶交叉组合，$x_i$表示特征值，$v_i$表示维度为$k$的隐向量。假设$m=n(n-1)/2$，则$f_{PI}(\xi)\in \R^{k\times m}$。

其实PI和BI的交叉过程完全一样，但是BI在交叉完了之后的pooling操作为等权求和处理，这是反直觉的，因为不同的交叉特征应该有着不同的重要性。不重要的交叉特征应该降低其权重，而重要性高的交叉特征应该提高其权重。Attention概念与该思想不谋而合，AFM作者顺势将其引入到模型之中，为每个交叉特征引入重要性权重，最终在对特征向量进行sum pooling时，利用重要性权重对二阶交叉特征进行加权求和。

为了计算特征重要性，专门构建了一个Attention Netowrk，表达式如下：

$$
\begin{align}
a_{i,j}&=\frac{exp({a'}_{i,j})}{\Sigma_{(i,j)\in \R_x}exp({a'}_{i,j})}\\
{a'}_{i,j}&=h^TReLU(W(v_i\odot v_j)x_ix_j+b)\\
\end{align}
$$

其中，$W\in \R^{t\times k}$，$b\in \R^t$，$h\in \R^t$是模型参数。参数$t$是attention节点数，计算得到的$a_{i,j}$也即对应二阶交叉特征$(v_i\odot v_j)x_ix_j$的权重，之后根据权重进行加权sum pooling。其结果仍为一个向量。

## 模型训练
因为又加入了attention参数，最终总的参数为$\Theta=\{w_0,\{w_i\}_{i=1}^n,\{v_i\}_{i=1}^n,p,W,b,h\}$，参数量的增加很容易导致过拟合。除了像NFM一样加BN和dropout，在损失中也加入了L2正则项。